In [1]:

# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [3]:

# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
        
        
        
        
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale, Wexford"
1,M6P,West Toronto,High Park
2,M5A,Downtown Toronto,Harbourfront
3,M1J,Scarborough,Scarborough Village
4,M4L,East Toronto,India Bazaar
5,M6M,York,"Keelesdale, Mount Dennis, Silverthorn"
6,M9N,York,Weston
7,M6H,West Toronto,Dovercourt Village
8,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
9,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."


In [6]:
df_toronto.shape

(77, 3)

In [7]:
!wget -O "coordinates.csv" https://cocl.us/Geospatial_data
coordinates = pd.read_csv("coordinates.csv")

--2020-01-13 16:17:23--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-13 16:17:24--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-13 16:17:24--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2sm

In [8]:
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:

print("coordinates shape: ", coordinates.shape)
print("dataset shape: ", df_toronto.shape)

coordinates shape:  (103, 3)
dataset shape:  (77, 3)


In [15]:
df = pd.merge(df_toronto, coordinates, left_on = 'Postcode', right_on = 'Postal Code')
df.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1R,Scarborough,"Maryvale, Wexford",M1R,43.750072,-79.295849
1,M6P,West Toronto,High Park,M6P,43.661608,-79.464763
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
4,M4L,East Toronto,India Bazaar,M4L,43.668999,-79.315572


In [16]:
print("dataset shape: ", df.shape)
df.to_csv("data_Toronto.csv")

dataset shape:  (77, 6)


In [17]:
!conda install -c conda-forge folium=0.5.0
!conda install -c conda-forge geopy

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [18]:
import folium # map-rendering library
import requests # library to handle requests
import json
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize

In [19]:

print("Stats Overview")
print("Borough: ", len(df['Borough'].unique()))
print("Neighborhoods: ", df.shape[0])

Stats Overview
Borough:  9
Neighborhoods:  77


In [20]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for borough, neighbor, latitude, longitude in zip(df['Borough'], df['Neighbor'], df['Latitude'], df['Longitude']):
    label = '{}, {}'.format(neighbor, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_Toronto)
map_Toronto

KeyError: 'Neighbor'

In [24]:
CLIENT_ID = '4NEWELJI2LL05CRZYN2ROXXQXOO1HTL2KWR0BQUB3AQD4WU2' # your Foursquare ID
CLIENT_SECRET = 'WWJDGBVEJDPO4L2UKVSAXOJFA41CXF5TCHV2H33XZJBUP3NZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4NEWELJI2LL05CRZYN2ROXXQXOO1HTL2KWR0BQUB3AQD4WU2
CLIENT_SECRET:WWJDGBVEJDPO4L2UKVSAXOJFA41CXF5TCHV2H33XZJBUP3NZ


In [25]:
neighborhood_name = df.loc[0,'Neighbor']
neighborhood_latitude = df.loc[0,'Latitude']
neighborhood_longitude = df.loc[0, 'Longitude']
print("The first neighborhood is ", neighborhood_name)
print("Its latitude is {} and longitude is {}".format(neighborhood_latitude, neighborhood_longitude))

KeyError: 'Neighbor'

In [26]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

NameError: name 'neighborhood_latitude' is not defined

In [27]:
results = requests.get(url).json()

NameError: name 'url' is not defined

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'results' is not defined

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
print('Sadly there are not a lot of places for people to go...')

NameError: name 'nearby_venues' is not defined

In [31]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
central_data = df[df['Borough'] == 'Central Toronto']
central_data

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
8,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,43.686412,-79.400049
33,M5R,Central Toronto,"The Annex, Yorkville",M5R,43.672710,-79.405678
35,M4T,Central Toronto,Moore Park,M4T,43.689574,-79.383160
36,M5P,Central Toronto,Forest Hill North,M5P,43.696948,-79.411307
37,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [33]:
central_venues = getNearbyVenues(names=central_data['Neighbor'],
                                   latitudes=central_data['Latitude'],
                                   longitudes=central_data['Longitude']
                                  )

KeyError: 'Neighbor'

In [34]:
print(central_venues.shape)
central_venues.groupby('Neighborhood').count()

NameError: name 'central_venues' is not defined

In [35]:
# one hot encoding
central_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_onehot['Neighborhood'] = central_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])
central_onehot = central_onehot[fixed_columns]

central_onehot.head()

NameError: name 'central_venues' is not defined

In [36]:

central_onehot = central_onehot.groupby('Neighborhood').mean().reset_index()
central_onehot.head()

NameError: name 'central_onehot' is not defined

In [37]:

num_top_venues = 5

for hood in central_onehot['Neighborhood']:
    print("----"+hood+"----")
    temp = central_onehot[central_onehot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'central_onehot' is not defined

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_onehot['Neighborhood']

for ind in np.arange(central_onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

NameError: name 'central_onehot' is not defined

In [40]:
 set number of clusters
kclusters = 5

central_cluster = central_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

SyntaxError: invalid syntax (<ipython-input-40-7cb04b2272f0>, line 1)

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
central_merged = df[df["Borough"] == 'Central Toronto']
central_merged = central_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbor')

central_merged.head() # check the last columns!

NameError: name 'kmeans' is not defined

In [42]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_merged['Latitude'], central_merged['Longitude'], central_merged['Neighbor'], central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'kclusters' is not defined

In [43]:
central_merged.loc[central_merged['Cluster Labels'] == 0, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

NameError: name 'central_merged' is not defined

In [44]:
central_merged.loc[central_merged['Cluster Labels'] == 1, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

NameError: name 'central_merged' is not defined

In [45]:
central_merged.loc[central_merged['Cluster Labels'] == 2, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

NameError: name 'central_merged' is not defined

In [46]:
central_merged.loc[central_merged['Cluster Labels'] == 3, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

NameError: name 'central_merged' is not defined

In [47]:
central_merged.loc[central_merged['Cluster Labels'] == 4, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

NameError: name 'central_merged' is not defined